<div align="center" style="padding: 28px; border: 2px solid #e7eaf6; border-radius: 18px; background: linear-gradient(135deg,#f8fbff, #fff7fd); box-shadow: 0 10px 30px rgba(0,0,0,0.06);">



  <h2 style="margin: 0; font-weight: 600;">
    🤖 LLM Agent
  </h2>

  <hr style="border: none; height: 1px; background: #e7eaf6; width: 82%; margin: 14px 0 10px;" />

  <p style="font-size: 1.12em; margin: 0.3em 0 0;">
    <strong>Ashkan Yousefnia</strong>
  </p>



  <p style="font-size: 0.98em; margin: 0.6em 0 0; opacity: 0.75;">
    <em>Exploring intelligent agents powered by large language models.</em>
  </p>

</div>

<!-- Plain Markdown fallback if HTML styles are stripped:
# Deep Learning Project
## LLM Agent
**Ashkan Yousefnia**  
Student ID: `401102829`
-->


In [1]:
from openai import OpenAI
import requests
import fitz
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import nltk
import faiss
import numpy as np
metis_api_key='tpsg-UMjoKAcsNZZ1u9YHcIyFlCLnh2zlT02'
import re
import os
# import msvcrt
# import keyboard  
from types import SimpleNamespace

client = OpenAI(api_key = metis_api_key, base_url="https://api.metisai.ir/openai/v1")

This Agent uses local Tokenizer but the model is used via api call(there is a version using Mistral model at the end of this code)

In [ ]:

class Agent:
    def __init__(self, api_token="tpsg-UMjoKAcsNZZ1u9YHcIyFlCLnh2zlT02"):
        self.api_token = api_token
        self.model = "gpt-4.1-mini"
        self.client = OpenAI(api_key=self.api_token, base_url="https://api.metisai.ir/openai/v1")
        self.conversation = []  # List of (role, content)
        self.summary = None
        self.max_turns_before_summarizing = 2  # Number of user-assistant pairs
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def capture_the_flag(self, prompt: str) -> str:
        # Add current user prompt to conversation
        reply = ""
        self.conversation.append(("user", prompt))
        
        
        # Check if summarization is needed
        
        if self.needs_summarization():
            self.update_memory()




        # Build prompt with context
        if self.check_if_pdf_input(prompt):
            
             
            path=self.extract_local_pdf_paths(prompt)
            chunks,embeddings=self.pdf_address_to_chunk_embedding(address=path)
            index, chunks=self.build_or_load_faiss_index(chunks=chunks,embeddings=embeddings)


            query = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": f"I am going to give you a prompt. After reading the prompt write a single query based on the given prompt so that I can look for the answer of that query in the pdf file. prompt:-{prompt}-. Just print the query."}],
                max_tokens=300,
                temperature=0.7
            )

            query = query.choices[0].message.content.strip()

            

            results = self.search_faiss_index(index, chunks, query)
            

            reply = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": f"This is the query that I have: {query}.These are the relevant answers I found based on the query: {results}. Now based on the provided information give a answer to the query."}],
                max_tokens=300,
                temperature=0.7
            )
            reply = reply.choices[0].message.content.strip()
            return reply

        elif self.game_mode(prompt=prompt):
            try:
                play()
            except SystemExit as e:
                # Avoid printing traceback in Jupyter when exiting with a message
                if isinstance(e.code, str):
                    print(e.code)
                raise  # re-raise so IPython doesn't crash with weird traceback
            except KeyboardInterrupt:
                print("\nGame aborted. Bye!")
            

        elif self.check_for_web_search(prompt=prompt):

            reply=self.web_searcher(prompt)

            self.conversation.append(("assistant", reply))
            return reply
            
        
        else:
            messages = self.build_prompt(prompt)

            # Call model
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                max_tokens=300,
                temperature=0.7
            )

            reply = response.choices[0].message.content.strip()
            self.conversation.append(("assistant", reply))

        return reply

    def needs_summarization(self) -> bool:
        # If too many total messages (user+assistant), trigger summarization
        return len(self.conversation) > self.max_turns_before_summarizing * 2

    def update_memory(self):
        # Summarize everything except the most recent few turns
        old_convo = self.conversation[:-self.max_turns_before_summarizing]
        if not old_convo:
            return

        convo_text = "\n".join(f"{role.capitalize()}: {content}" for role, content in old_convo)

        summary_prompt = [
            {"role": "system", "content": "You are a summarizer."},
            {"role": "user", "content": f"Summarize the following conversation for memory:\n\n{convo_text}"}
        ]

        response = self.client.chat.completions.create(
            model=self.model,
            messages=summary_prompt,
            max_tokens=200,
            temperature=0.3
        )

        self.summary = response.choices[0].message.content.strip()

        # Retain only the recent portion of the conversation
        self.conversation = self.conversation[-self.max_turns_before_summarizing:]

    def build_prompt(self, current_user_input: str):

        messages = []
        
        if self.summary:
            messages.append({
                "role": "system",
                "content": f"Summary of prior conversation: {self.summary}"
            })

        # Add recent conversation history
        for role, content in self.conversation:
            messages.append({"role": role, "content": content})

        # Add current prompt again as the latest user message
        # (optional if already added in self.conversation)
        # messages.append({"role": "user", "content": current_user_input})

        return messages

    def check_for_web_search(self,prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{prompt}. if you need more data to answer user question only say SEARCH."}] ,
            max_tokens=100,
            temperature=0.1
        )
        response = response.choices[0].message.content.strip()

        #print(response)
        if response == 'SEARCH':
            return True
        else:
            return False

    def web_searcher(self,prompt):

        num_results=3
        #message=self.build_prompt(prompt)
        query = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"in order to answer the following prompt write the best query so i can search the internet with that query {prompt}. just write the query"}],
            max_tokens=100,
            temperature=0.1
        )
        query=query.choices[0].message.content.strip()
        
        EXA_API_KEY = "679619bb-2cb6-4e88-8ee1-33febc5ffbac"
        EXA_ENDPOINT = "https://api.exa.ai/search"

        headers = {
        "Content-Type": "application/json",
        "x-api-key": EXA_API_KEY
        }

        

        payload = {
            "query": query,
            "num_results": num_results,
            "include_domains": [],
            "exclude_domains": [],
            "use_autoprompt": True,   # optional for better queries
            "type": "auto"            
        }

        response = requests.post(EXA_ENDPOINT, json=payload, headers=headers,proxies={})
        response.raise_for_status()

        results = response.json()["results"]
        urls = [item["url"] for item in results]
        contents=[]
        headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
        }   
        
        for url in urls:
            response = requests.get(url,headers=headers)
            response.raise_for_status()  # raises HTTPError if status is 4xx or 5xx
            content = response.text
            contents.append(content)

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{query}.for the query that i just gave you i have searched the internet and i am going to provide you with the content of the three most relevant results.find the best answer for the query in the contents of these web sites  {contents}. just write the answer"}],
            max_tokens=100,
            temperature=0.1
        )
        
        response=response.choices[0].message.content.strip()

        return response
    
    
    def pdf_address_to_chunk_embedding(self,address,chunk_len=250):
        
        
       
        nltk.download('punkt_tab')
  

        doc = fitz.open(address)
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()

        # Step 2: Tokenize and chunk into 100-word chunks
        words = word_tokenize(full_text)
        chunks = [' '.join(words[i:i + chunk_len])
                  for i in range(0, len(words), chunk_len)]
        # compute embeddings as numpy float32 array
        embeddings = self.embedding_model.encode(chunks)
        embeddings = np.asarray(embeddings, dtype='float32')

        return chunks,embeddings
    


    def build_or_load_faiss_index(self,chunks, embeddings, index_path="faiss.index"):
       
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)                 # now a proper numpy array
        faiss.write_index(index, index_path)
        return index, chunks

    def search_faiss_index(self,index, chunks, query, top_k=3):

        q_emb = self.embedding_model.encode([query])
        q_emb = np.asarray(q_emb, dtype='float32')
        distances, indices = index.search(q_emb, top_k)
        print(indices)
        return [chunks[i] for i in indices[0]]

    def check_if_pdf_input(self,query):
        return '.pdf' in query.lower()
    
    def extract_local_pdf_paths(self,text):
        # Match file paths with spaces, until .pdf
        pattern = r'([A-Za-z]:[\\/](?:[^:*?"<>|\r\n]+[\\/])*[^:*?"<>|\r\n]+\.pdf)'
        matches = re.findall(pattern, text)
        pdf_paths = [match.strip() for match in matches if os.path.exists(match.strip())]
        return pdf_paths[0] if pdf_paths else None
    def game_mode(self,prompt):

        # response = self.client.chat.completions.create(
        #     model=self.model,
        #     messages=[{"role": "user", "content": f"Read this user input: -{prompt}-.If user input contain words like: game/20 question/20 question game/guess, just print : GAME"}],
        #     max_tokens=100,
        #     temperature=0.1
        # )
        
        # response=response.choices[0].message.content.strip()

        # if response=="GAME":
        #     return True
        
        # return False
        return '20 question game' in prompt.lower()

    







The code that is related to the 20 question game:

In [4]:
import json
import os
import pandas as pd
import random
import string
import torch

from keywords import KEYWORDS_JSON
from os import path
from pathlib import Path
from random import choice
from string import Template
from transformers import T5Tokenizer, T5ForConditionalGeneration


#llm_parent_dir = "/kaggle/input/flan-t5/pytorch/large"

device = None
model = None
tokenizer = None
model_initialized = False

ERROR = "ERROR"
DONE = "DONE"
INACTIVE = "INACTIVE"
ACTIVE = "ACTIVE"
TIMEOUT = "TIMEOUT"

GUESS = "guess"
ASK = "ask"
GUESSER = "guesser"
ANSWERER = "answerer"

RANDOM_GUESSER = "random_guesser"
RANDOM_ANSWERER = "random_answerer"

def weighted_random_category(keywords_list):
    try:
        return random.choices(population=keywords_list, weights=[len(entry["words"]) for entry in keywords_list], k=1)[0]
    except:
        pass
    return random.choice(keywords_list)

keywords_list = json.loads(KEYWORDS_JSON)
keyword_cat = weighted_random_category(keywords_list)
category = keyword_cat["category"]
keyword_obj = random.choice(keyword_cat["words"])
keyword = keyword_obj["keyword"]
alts = keyword_obj["alts"]

try:
    with open("/kaggle_simulations/data.json") as f:
        json_content = f.read()
        d_keywords_list = json.loads(json_content)
        d_keyword_cat = weighted_random_category(d_keywords_list)
        d_category = d_keyword_cat["category"]
        d_keyword_obj = random.choice(d_keyword_cat["words"])
        d_keyword = d_keyword_obj["keyword"]
        d_alts = d_keyword_obj["alts"]
        # re-assign
        category = d_category
        keyword = d_keyword
        alts = d_alts
except:
    pass


def random_guesser(obs):
    if obs.turnType == GUESS:
        return "banana"
    if random.random() < .5:
        return "Is is a person?"
    if random.random() < .5:
        return "Is it a place?"
    return "Is it a thing?"

def random_answerer():
    if random.random() > .5:
        return "yes"
    return "no"


def guesser_agent(obs):
    info_prompt = """You are playing a game of 20 questions where you ask the questions and try to figure out the keyword, which will be a real or fictional person, place, or thing. \nHere is what you know so far:\n{q_a_thread}"""
    questions_prompt = """Ask one yes or no question."""
    guess_prompt = """Guess the keyword. Only respond with the exact word/phrase. For example, if you think the keyword is [paris], don't respond with [I think the keyword is paris] or [Is the keyword Paris?]. Respond only with the word [paris]."""

    q_a_thread = ""
    for i in range(0, len(obs.answers)):
        q_a_thread = "{}Q: {} A: {}\n".format(
            q_a_thread,
            obs.questions[i],
            obs.answers[i]
        )

    prompt = ""
    if obs.turnType == ASK:
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            questions_prompt
        )
    elif obs.turnType == GUESS:
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            guess_prompt
        )
    else:
        return ""
    
    return call_llm(prompt)

    

def answerer_agent(obs):
    info_prompt = """You are a very precise answerer in a game of 20 questions. The keyword that the questioner is trying to guess is [the {category} {keyword}]. """
    answer_question_prompt = """Answer the following question with only yes, no, or if unsure maybe: {question}"""

    if obs.turnType == "answer":
        prompt = "{}{}".format(
            info_prompt.format(category=category,keyword=keyword),
            answer_question_prompt.format(question=obs.questions[-1])
        )
        return call_llm(prompt)
    else: 
        return ""


agents = {GUESSER: guesser_agent, ANSWERER: answerer_agent, RANDOM_ANSWERER: random_answerer, RANDOM_GUESSER: random_guesser}

def guesser_action(active, inactive, step):
    inactive.observation.keyword = keyword
    inactive.observation.category = category
    guessed = False
    bad_guess = False
    if not active.action:
        active.status = ERROR
        bad_guess = True
    elif active.observation.turnType == ASK:
        question = active.action[:750]
        active.observation.questions.append(question)
        inactive.observation.questions.append(question)
    elif active.observation.turnType == GUESS:
        guess = active.action[:100]
        active.observation.guesses.append(guess)
        inactive.observation.guesses.append(guess)
    if active.action and keyword_guessed(active.action):
        guessed = True
        score = 20 - int(step / 3)
        end_game(active, score, DONE)
        end_game(inactive, score, DONE)
    return [guessed, bad_guess]

def end_game(agent, reward, status):
    agent.observation.keyword = keyword
    agent.observation.category = category
    agent.reward = reward
    agent.status = status

def answerer_action(active, inactive):
    active.observation.keyword = keyword
    active.observation.category = category
    response = active.action
    bad_response = False
    if not response:
        response = "none"
        end_game(active, -1, ERROR)
        end_game(inactive, 1, DONE)
        bad_response = True
    elif "yes" in response.lower():
        response = "yes"
    elif "no" in response.lower():
        response = "no"
    else:
        response = "maybe"
        end_game(active, -1, ERROR)
        end_game(inactive, 1, DONE)
        bad_response = True
    active.observation.answers.append(response)
    inactive.observation.answers.append(response)
    return bad_response

def increment_turn(active, inactive, step, guessed):
    if step == 59 and not guessed:
        end_game(active, -1, DONE)
        end_game(inactive, -1, DONE)
    elif active.observation.turnType == "guess":
        active.observation.turnType = "ask"
    elif active.observation.turnType == "ask":
        active.observation.turnType = "guess"
        active.status = INACTIVE
        inactive.status = ACTIVE
    else:
        active.status = INACTIVE
        inactive.status = ACTIVE


def interpreter(state, env):
    if env.done:
        return state

    # Isolate the active and inactive agents.
    active1 = state[0] if state[0].status == ACTIVE else state[1]
    inactive1 = state[0] if state[0].status == INACTIVE else state[1]
    active2 = state[2] if state[2].status == ACTIVE else state[3]
    inactive2 = state[2] if state[2].status == INACTIVE else state[3]
    if active1.status == DONE and inactive1.status == DONE:
        active1 = None
        inactive1 = None
    if active2.status == DONE or inactive2.status == DONE:
        active2 = None
        inactive2 = None
    if active1 is None and inactive1 is None and active2 is None and inactive2 is None:
        return state

    step = state[0].observation.step

    end_early = (active1 and active1.status) in (TIMEOUT, ERROR) or (active2 and active2.status in (TIMEOUT, ERROR))
    one_guessed = False
    two_guessed = False
    one_bad_guess = False
    two_bad_guess = False
    one_bad_response = False
    two_bad_response = False

    if active1 is None or active2 is None:
        raise ValueError

    if active1.observation.role == GUESSER:
        [one_guessed, one_bad_guess] = guesser_action(active1, inactive1, step)
    else:
        one_bad_response = answerer_action(active1, inactive1)

    if active2.observation.role == GUESSER:
        [two_guessed, two_bad_guess] = guesser_action(active2, inactive2, step)
    else:
        two_bad_response = answerer_action(active2, inactive2)

    if active1.status in (TIMEOUT, ERROR) or one_bad_response or one_bad_guess:
        end_game(active1, -1, active1.status)
        end_game(inactive1, 1, DONE)
    elif end_early or two_bad_response or two_bad_guess:
        end_game(active1, 1, DONE)
        end_game(inactive1, 1, DONE)
    else:
        increment_turn(active1, inactive1, step, one_guessed)
    
    if active2.status in (TIMEOUT, ERROR) or two_bad_response or two_bad_guess:
        end_game(active2, -1, active2.status)
        end_game(inactive2, 1, DONE)
    elif end_early or one_bad_response or one_bad_guess:
        end_game(active2, 1, DONE)
        end_game(inactive2, 1, DONE)
    else:
        increment_turn(active2, inactive2, step, two_guessed)
    
    # make sure to end the game if only one team guessed correctly this round
    if one_guessed and not two_guessed:
        end_game(active2, 0, DONE)
        end_game(inactive2, 0, DONE)
    elif two_guessed and not one_guessed:
        end_game(active1, 0, DONE)
        end_game(inactive1, 0, DONE)
    
    return state


def renderer(state, env):

    for s in state:
        print("role: ", s.observation.role)
        if s.observation.role == GUESSER:
            transcript = ""
            for i in range(0, len(s.observation.guesses)):
                transcript = "{}Q: {} A: {}\nG: {}\n".format(
                    transcript, s.observation.questions[i],
                    s.observation.answers[i],
                    s.observation.guesses[i]
                )
            print(transcript)

        print("keyword: ", s.observation.keyword)
        print("score: ", s.reward)
        print("")
        print("")
        print("")

    return ""


jsonpath = os.path.abspath(os.path.join(os.getcwd(), "llm_20_questions.json"))
with open(jsonpath) as f:
    specification = json.load(f)

def html_renderer():
    jspath = path.abspath(path.join(path.dirname(__file__), "llm_20_questions.js"))
    with open(jspath, encoding="utf-8") as f:
        return f.read()


def normalize(s: str) -> str:
    t = str.maketrans("", "", string.punctuation)
    return s.lower().replace("the", "").replace(" ", "").translate(t)

def compare_words(a, b) -> bool:
    a = normalize(a)
    b = normalize(b)
    if a == b:
        return True
    # don't check for plurals if string is too short
    if len(a) < 3 or len(b) < 3:
        return False
    # accept common plurals
    if a[-1] == "s" and a[:-1] == b:
        return True
    if b[-1] == "s" and a == b[:-1]:
        return True
    if a[-2:] == "es" and a[:-2] == b:
        return True
    if b[-2:] == "es" and a == b[:-2]:
        return True
    return False

def keyword_guessed(guess: str) -> bool:
    if compare_words(guess, keyword):
      return True
    for s in alts:
      if compare_words(s, guess):
        return True
    return False


def call_llm(prompt: str) -> str:
    global model_initialized
    global device
    global model
    global tokenizer

   
        
    

    # inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # outputs = model.generate(**inputs)
    # answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    api_token="tpsg-7SfPTJFEkQZNDYc7NjLOZ3c34UDuCqM"
    api_token = api_token
    model = "gpt-4.1-mini"
    client = OpenAI(api_key=api_token, base_url="https://api.metisai.ir/openai/v1")
    answer = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.7
            )
    answer = answer.choices[0].message.content.strip()
    return answer


In [5]:
#!/usr/bin/env python3
"""
play_20_questions.py

Driver for 20Q with *two* distinct guess-lists:
  - question_guesses: the agent’s prior keyword guesses
  - answer_guesses:   your prior yes/no/maybe replies

Both lists are passed into the LLM every time so it can use all history.
This enhanced version maintains a normalized set of past guesses to prevent duplicates,
and preserves the original guess text (including casing) when displayed.
"""

import sys
from types import SimpleNamespace

# from llm_20_questions import guesser_agent, ASK, GUESS

# Constants for turn types
ASK = "ask"
GUESS = "guess"


def normalize_guess(s: str) -> str:
    """
    Normalize a guess string for comparison (case-folded, stripped).
    """
    return s.casefold().strip()


def play():
    questions = []
    answer_guesses = []
    question_guesses = []
    past_guesses_norm = set()  # normalized past guesses
    qnum = 1

    print("\nWelcome! Think of a person/place/thing; I'll try to guess it in 20 questions.\n")
    print("Type 'exit' at any time to quit the game.\n")

    while qnum <= 20:
        # ASK turn --------------------------------------------------------
        obs = SimpleNamespace(
            turnType=ASK,
            questions=questions,
            answers=answer_guesses,
            question_guesses=question_guesses,
            answer_guesses_list=answer_guesses,
        )
        question = guesser_agent(obs).strip()
        print(f"Q{qnum}. {question}")
        sys.stdout.flush()
        questions.append(question)

        # user's yes/no/maybe reply
        ans = ""
        while ans.lower() not in ("yes", "no", "maybe"):
            ans = input("Your answer (yes/no/maybe): ").strip()
            if ans.lower() == "exit":
                sys.exit("\nGame aborted by user. Bye!")
        answer_guesses.append(ans.lower())

        # GUESS turn ------------------------------------------------------
        obs = SimpleNamespace(
            turnType=GUESS,
            questions=questions,
            answers=answer_guesses,
            question_guesses=question_guesses,
            answer_guesses_list=answer_guesses,
        )
        # Request and validate a new guess
        tries = 0
        guess = guesser_agent(obs).strip()
        norm = normalize_guess(guess)
        # Avoid duplicates by normalized comparison
        

        # Record this guess
        question_guesses.append(guess)
        past_guesses_norm.add(norm)
        guess_idx = len(question_guesses)
        print(f"\nGuess #{guess_idx}: {guess}")
        sys.stdout.flush()

        # was it right?
        ok = ""
        while ok.lower() not in ("yes", "no"):
            try:
                ok = input("Did I get it right? (yes/no): ").strip()
                if ok.lower() == "exit":
                    sys.exit("\nGame aborted by user. Bye!")
            except (EOFError, KeyboardInterrupt):
                sys.exit("\nGame aborted. Bye!")
        if ok.lower() == "yes":
            print(f"\n🎉 Hooray! I got it in {guess_idx} guesses.")
            return

        # next round
        qnum += 1
        print("\n" + "— " * 10 + "\n")

    print("\n😔 I couldn't guess it within 20 questions. Well played!")






# Testing the Agent

In [28]:
agent=Agent()


# 1.Agent's History and summarization of history

In [39]:
ans=agent.capture_the_flag('hi how are you?')
print(ans)


Hello! I'm just a language model AI, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [30]:
print(agent.conversation)

[('user', 'hi how are you?'), ('assistant', "Hello! I'm doing great, thank you. How can I assist you today?")]


In [40]:
ans=agent.capture_the_flag('hi my name is Ashkan')
print(ans)

Hi Ashkan! Nice to meet you. How can I assist you today?


In [41]:
ans=agent.capture_the_flag('what is your job?')
print(ans)

My job is to assist you by answering questions, providing information, helping with writing, brainstorming ideas, explaining concepts, and much more. Basically, I'm here to help you with a wide range of tasks using natural language. How can I help you today, Ashkan?


In [30]:
print(agent.conversation)

[('assistant', 'Hi Ashkan! Nice to meet you. How can I help you today?'), ('user', 'what is your job?'), ('assistant', "My job is to assist you by providing information, answering questions, helping with creative writing, offering explanations, and supporting you in a wide range of topics. Basically, I'm here to help make your tasks easier and provide useful, accurate, and friendly assistance. How can I assist you today?")]


In [8]:
ans=agent.capture_the_flag('what are your abilities?')
print(ans)

I don’t know your name unless you tell me. 

As for my abilities, I can help with a wide range of tasks including:

- Answering questions and providing explanations on many topics
- Assisting with writing, editing, and brainstorming
- Offering tutoring and educational support
- Creating summaries and outlines
- Generating ideas for projects, stories, or other creative work
- Helping with coding and debugging
- Translating languages
- Providing recommendations and advice

Let me know how I can assist you!


In [34]:
print(agent.conversation)

[('assistant', 'Hi Ashkan! Nice to meet you. How can I help you today?'), ('user', 'what is your job?'), ('assistant', 'My job is to assist you by answering questions, providing information, helping with tasks, generating ideas, and engaging in conversations on a wide range of topics. How can I assist you today, Ashkan?')]


In [35]:
print(agent.summary)

The user greeted the assistant and introduced themselves as Ashkan.


In [42]:
ans=agent.capture_the_flag('what is my name?')
print(ans)

Your name is Ashkan.


In [38]:
print(agent.conversation)

[('user', 'Where is Iran located'), ('assistant', 'Iran is located in Western Asia, on the eastern side of the Middle East. It is bordered by several countries: to the northwest by Armenia and Azerbaijan, to the north by the Caspian Sea, to the northeast by Turkmenistan, to the east by Afghanistan and Pakistan, to the south by the Persian Gulf and the Gulf of Oman, and to the west by Turkey and Iraq. Its strategic location connects the Middle East with Central Asia and South Asia.'), ('user', 'Who was Adolf Hitler?'), ('assistant', "Adolf Hitler was a German political leader who became the dictator of Germany from 1934 to 1945. He was the leader of the National Socialist German Workers' Party (Nazi Party) and played a central role in the events leading up to and during World War II. Hitler promoted a totalitarian regime based on fascist and nationalist ideologies, including aggressive expansionism and racist policies, most notoriously anti-Semitism, which led to the Holocaust—the syste

In [39]:
print(agent.summary)

The user greeted the assistant and asked about its job and abilities. The assistant explained that its role is to assist by providing information, answering questions, and helping with various tasks. It listed abilities such as answering factual questions, aiding with writing and editing, explaining concepts, assisting with problem-solving, generating creative content, and offering recommendations. The assistant invited the user to specify any particular help needed.


Now here is the conversation summarization which happens after each 5 prompt and is given to the model with new instruction to help it remember the context

In [27]:
print(agent.summary)

None


# 2.Retrieval-Augmented Generation

Now in order to test my Agent's ability to process pdf files I gave it the Large Language Diffusion Models paper which exists in the the same folder.

IMPORTANT: Inorder to not get an error please change the address of the given paper accordingly

In [43]:
ans=agent.capture_the_flag('what is the title of this paper d://uni stuff//Semester 6//Deep Learning//Project//Large Language Diffusion Models.pdf')
print(ans)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[[45 31 51]]
The title of the paper in the file "Large Language Diffusion Models.pdf" is "Large Language Diffusion Models".


In [44]:
ans=agent.capture_the_flag('can you give a summary of this paper d://uni stuff//Semester 6//Deep Learning//Project//Large Language Diffusion Models.pdf')
print(ans)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[[45 41 64]]
The paper titled "Large Language Diffusion Models" explores the application of diffusion models within the realm of large language models (LLMs). Unlike traditional autoregressive or masked language models, diffusion-based approaches model the generation process as a gradual denoising or refinement of text sequences, offering a novel perspective for language modeling.

Key insights from the paper and related works include:

1. **Diffusion Models for Text**: The paper discusses adapting diffusion processes—originally successful in continuous domains like images—to discrete textual data, addressing challenges such as discrete token representation and score matching for discrete distributions.

2. **Modeling Conditional Distributions**: Techniques such as discrete diffusion with self-conditioning (e.g., "Text-to-text self-conditioned simplex diffusion") enable the model to better capture conditional dependencies in language generation tasks.

3. **Performance and Versatility*

In [56]:
ans=agent.capture_the_flag('Who wrote the paper d://uni stuff//Semester 6//Deep Learning//Project//Large Language Diffusion Models.pdf')
print(ans)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[[45 31 51]]
The authors of the paper titled "Large Language Diffusion Models" are:

Hugo Touvron, Louis Martin, Kevin Stone, Pierre Albert, Abdelrahman Almahairi, Younes Babaei, Nikita Bashlykov, Shagun Batra, Pranav Bhargava, Shubham Bhosale, et al.


# 3.Function Calling

# 3.1 Web Search Trigger

IMPORTANT: If your query reaches an error it means one of the web pages that the model wants to access is either filtered or doesn't allow Iranian IP

In [45]:
agent.check_for_web_search('What time is it in Iran right now?')

True

In [46]:
ans=agent.capture_the_flag('What time is it in Iran right now?')
print(ans)

The current local time in Iran (Tehran) is 20:21:50 IRST (Iran Standard Time), which is UTC +3:30. Iran does not observe daylight saving time as of 2025.


In [48]:
agent.check_for_web_search('who is the current president of Iran right now?')

False

In [47]:
ans=agent.capture_the_flag('who is the current president of Iran right now?')
print(ans)

As of June 2024, the current president of Iran is Ebrahim Raisi. He has been in office since August 2021.


In [60]:
agent.check_for_web_search('Who won the last chess Esport tournament and where was it held')


True

In [62]:
ans=agent.capture_the_flag('Who won the last chess Esport tournament and where was it held')
print(ans)

Magnus Carlsen won the latest chess esports tournament, the first ever edition of the Chess Esports World Cup. The tournament was held in Riyadh.


In [64]:
ans=agent.capture_the_flag('What was the result of the last Manchester United game and which team was it against')
print(ans)

Manchester United's last game result was a 0-1 loss against Arsenal on Sunday, August 17, 2025.


# 3.2 The 20 Question Game

IMPORTANT : inorder to go into the game your prompt must cantain 20 question game and you should type your answer as yes/no/maybe

type exit to quit the game

First Word : Friedrich Nietzsche 

In [8]:
agent.capture_the_flag('lets play the 20 question game')


Welcome! Think of a person/place/thing; I'll try to guess it in 20 questions.

Type 'exit' at any time to quit the game.

Q1. Is the keyword a living person?

Guess #1: Elon Musk

— — — — — — — — — — 

Q2. Is the keyword a fictional character?

Guess #2: Barack Obama

— — — — — — — — — — 

Q3. Is the keyword a currently living person?

Guess #3: Albert Einstein

— — — — — — — — — — 

Q4. Was the person primarily known for their work in the 20th century?

Guess #4: napoleon bonaparte

— — — — — — — — — — 

Q5. Was the person primarily known for their work before the 20th century?

Guess #5: leonardo da vinci

— — — — — — — — — — 

Q6. Was this person primarily known for their work in the 19th century?

Guess #6: abraham lincoln

— — — — — — — — — — 

Q7. Was this person primarily known for their contributions in the field of science or technology?

Guess #7: charles dickens

— — — — — — — — — — 

Q8. Was this person primarily known for their contributions in the field of politics or go

''

Second word : Lamp

In [10]:
agent.capture_the_flag('wanna play the 20 question game?')


Welcome! Think of a person/place/thing; I'll try to guess it in 20 questions.

Type 'exit' at any time to quit the game.

Q1. Is the keyword a person?

Guess #1: Eiffel Tower

— — — — — — — — — — 

Q2. Is the keyword a place?

Guess #2: car

— — — — — — — — — — 

Q3. Is the keyword a thing?

Guess #3: computer

— — — — — — — — — — 

Q4. Is the thing man-made?

Guess #4: smartphone

— — — — — — — — — — 

Q5. Is the thing commonly found indoors?

Guess #5: computer

— — — — — — — — — — 

Q6. Is the thing electronic?

Guess #6: smartphone

— — — — — — — — — — 

Q7. Is the electronic thing primarily used for communication?

Guess #7: television

— — — — — — — — — — 

Q8. Is the electronic thing primarily used for entertainment?

Guess #8: microwave

— — — — — — — — — — 

Q9. Is the electronic thing primarily used for work or productivity?

Guess #9: microwave

— — — — — — — — — — 

Q10. Is the electronic thing primarily used for household chores or maintenance?

Guess #10: calculator

— —

''

In [51]:
agent.capture_the_flag('lets play the 20 question game')


Welcome! Think of a person/place/thing; I'll try to guess it in 20 questions.

Type 'exit' at any time to quit the game.

Q1. Is the keyword a person?

Guess #1: eiffel tower

— — — — — — — — — — 

Q2. Is the keyword a place?

Guess #2: book

— — — — — — — — — — 

Q3. Is the keyword a fictional thing?

Guess #3: thing

— — — — — — — — — — 

Q4. Is the keyword a real thing?

Guess #4: computer

— — — — — — — — — — 

Q5. Is the keyword a living thing?

Guess #5: dog

— — — — — — — — — — 

Q6. Is the keyword an animal?

Guess #6: dog

— — — — — — — — — — 

Q7. Is the animal a mammal?

Guess #7: eagle

— — — — — — — — — — 

Q8. Is the animal a bird?

Guess #8: shark

🎉 Hooray! I got it in 8 guesses.


''

IMPORTANT : If the evaluate_20Q.py didn't work properly you can easily give the keywords to the model here and get accuracy

# 4.GUI

In [52]:


import gradio as gr



def run_ctf(prompt: str):
    try:
        return agent.capture_the_flag(prompt)
    except SystemExit as e:
        # In case game_mode/play() calls exit
        return str(e) if isinstance(e.code, str) else "Game ended."
    except Exception as e:
        return f"Error: {type(e).__name__}: {e}"

# ----- Styling (beauty only) -----
CSS = """
:root { --radius-xl: 18px; }
#header {
  background: linear-gradient(135deg,#111827 0%,#0ea5e9 50%,#10b981 100%);
  padding: 22px 18px; border-radius: var(--radius-xl); color: white;
  box-shadow: 0 10px 28px rgba(0,0,0,.25);
}
#header h2 { margin: 0 0 4px 0; font-size: 1.4rem; }
#header p { margin: 0; opacity: .95; }
#prompt_box textarea { min-height: 90px; font-size: 16px; }
#result_box textarea { min-height: 360px; font-size: 15px; }
footer { visibility: hidden; } /* cleaner look */
"""

theme = gr.themes.Soft(
    primary_hue="teal",
    secondary_hue="slate",
).set(
    block_shadow="*shadow_drop_lg",
    block_border_width="1px",
    button_primary_background_fill="*primary_600",
    button_primary_background_fill_hover="*primary_700",
    body_text_size="16px",
)

with gr.Blocks(title="CTF Agent (Minimal)", theme=theme, css=CSS) as demo:
    with gr.Group(elem_id="header"):
        gr.HTML("""
            <h2>🏴‍☠️ CTF Agent</h2>
            <p>Every input goes straight to <code>capture_the_flag()</code> — nothing else.</p>
        """)

    with gr.Row():
        prompt = gr.Textbox(
            label="Your Prompt",
            placeholder="Type anything. It will be sent directly to capture_the_flag().",
            autofocus=True,
            lines=2,
            elem_id="prompt_box",
        )

    result = gr.Textbox(
        label="Result",
        lines=16,
        interactive=False,
        elem_id="result_box",
        show_copy_button=True,
    )

    with gr.Row():
        send = gr.Button("➤ Send", variant="primary", scale=2)
        clear = gr.Button("🧹 Clear", variant="secondary")

    # Wire events (unchanged behavior)
    prompt.submit(run_ctf, inputs=prompt, outputs=result)
    send.click(run_ctf, inputs=prompt, outputs=result)
    clear.click(lambda: ("", ""), inputs=None, outputs=[prompt, result], queue=False)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# Agent with local Mistral

Note that for this part I downloaded the Mistral model on my laptop and ran the code locally through a 4090 gpu on server

In [1]:
# NEW: local Mistral client shim that mimics OpenAI's chat.completions.create
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sentence_transformers import SentenceTransformer


class _SimpleMessage:
    def __init__(self, content: str):
        self.content = content

class _SimpleChoice:
    def __init__(self, content: str):
        self.message = _SimpleMessage(content)

class _SimpleResponse:
    def __init__(self, content: str):
        self.choices = [_SimpleChoice(content)]

class _Completions:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def create(self, model, messages, max_tokens=300, temperature=0.7):
        # Build a chat-formatted prompt using the tokenizer's chat template
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        inputs = self.tokenizer(prompt, return_tensors="pt")
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            output_ids = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=temperature,
                pad_token_id=self.tokenizer.eos_token_id
            )

        generated = output_ids[0, inputs["input_ids"].shape[1]:]
        text = self.tokenizer.decode(generated, skip_special_tokens=True).strip()
        return _SimpleResponse(text)

class _Chat:
    def __init__(self, model, tokenizer):
        self.completions = _Completions(model, tokenizer)

class LocalMistralClient:
    def __init__(self, model_name: str = "mistralai/Mistral-7B-Instruct-v0.3"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Try to use 4-bit quantization if available; otherwise fall back to fp16.
        try:
            from transformers import BitsAndBytesConfig
            quant_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=quant_config,
                device_map="auto"
            )
        except Exception:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )

        # Expose an OpenAI-like surface
        self.chat = _Chat(self.model, self.tokenizer)


# ===== YOUR ORIGINAL CLASS, MINIMALLY MODIFIED IN __init__ ONLY =====

class Agent:
    def __init__(self, api_token="tpsg-7SfPTJFEkQZNDYc7NjLOZ3c34UDuCqM"):
        self.api_token = api_token
        self.model = "mistralai/Mistral-7B-Instruct-v0.3"
        # CHANGED: use local Mistral client instead of OpenAI API client
        self.client = LocalMistralClient(self.model)
        self.conversation = []  # List of (role, content)
        self.summary = None
        self.max_turns_before_summarizing = 2  # Number of user-assistant pairs
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def capture_the_flag(self, prompt: str) -> str:
        # Add current user prompt to conversation
        reply = ""
        self.conversation.append(("user", prompt))
        
        
        # Check if summarization is needed
        
        if self.needs_summarization():
            self.update_memory()

        

        # Build prompt with context
        if self.check_if_pdf_input(prompt):
            
             
            path=self.extract_local_pdf_paths(prompt)
            chunks,embeddings=self.pdf_address_to_chunk_embedding(address=path)
            index, chunks=self.build_or_load_faiss_index(chunks=chunks,embeddings=embeddings)


            query = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": f"I am going to give you a prompt. After reading the prompt write a single query based on the given prompt so that I can look for the answer of that query in the pdf file. prompt:-{prompt}-. Just print the query."}],
                max_tokens=300,
                temperature=0.7
            )

            query = query.choices[0].message.content.strip()

            

            results = self.search_faiss_index(index, chunks, query)
            

            reply = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": f"This is the query that I have: {query}.These are the relevant answers I found based on the query: {results}. Now based on the provided information give a answer to the query."}],
                max_tokens=300,
                temperature=0.7
            )
            reply = reply.choices[0].message.content.strip()

        elif self.game_mode(prompt=prompt):
            try:
                play()
            except SystemExit as e:
                # Avoid printing traceback in Jupyter when exiting with a message
                if isinstance(e.code, str):
                    print(e.code)
                raise  # re-raise so IPython doesn't crash with weird traceback
            except KeyboardInterrupt:
                print("\nGame aborted. Bye!")
            

        elif self.check_for_web_search(prompt=prompt):

            reply=self.web_searcher(prompt)

            self.conversation.append(("assistant", reply))
            
        
        else:
            messages = self.build_prompt(prompt)

            # Call model
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                max_tokens=300,
                temperature=0.7
            )

            reply = response.choices[0].message.content.strip()
            self.conversation.append(("assistant", reply))

        return reply

    def needs_summarization(self) -> bool:
        # If too many total messages (user+assistant), trigger summarization
        return len(self.conversation) > self.max_turns_before_summarizing * 2

    def update_memory(self):
        # Summarize everything except the most recent few turns
        old_convo = self.conversation[:-self.max_turns_before_summarizing]
        if not old_convo:
            return

        convo_text = "\n".join(f"{role.capitalize()}: {content}" for role, content in old_convo)

        summary_prompt = [
            {"role": "system", "content": "You are a summarizer."},
            {"role": "user", "content": f"Summarize the following conversation for memory:\n\n{convo_text}"}
        ]

        response = self.client.chat.completions.create(
            model=self.model,
            messages=summary_prompt,
            max_tokens=200,
            temperature=0.3
        )

        self.summary = response.choices[0].message.content.strip()

        # Retain only the recent portion of the conversation
        self.conversation = self.conversation[-self.max_turns_before_summarizing:]

    def build_prompt(self, current_user_input: str):

        messages = []
        
        if self.summary:
            messages.append({
                "role": "system",
                "content": f"Summary of prior conversation: {self.summary}"
            })

        # Add recent conversation history
        for role, content in self.conversation:
            messages.append({"role": role, "content": content})

        # Add current prompt again as the latest user message
        # (optional if already added in self.conversation)
        # messages.append({"role": "user", "content": current_user_input})

        return messages

    def check_for_web_search(self,prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{prompt}. if you need more data to answer user question only say SEARCH."}] ,
            max_tokens=100,
            temperature=0.1
        )
        response = response.choices[0].message.content.strip()

        #print(response)
        if response == 'SEARCH':
            return True
        else:
            return False

    def web_searcher(self,prompt):

        num_results=3
        #message=self.build_prompt(prompt)
        query = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"in order to answer the following prompt write the best query so i can search the internet with that query {prompt}. just write the query"}],
            max_tokens=100,
            temperature=0.1
        )
        query=query.choices[0].message.content.strip()
        
        EXA_API_KEY = "679619bb-2cb6-4e88-8ee1-33febc5ffbac"
        EXA_ENDPOINT = "https://api.exa.ai/search"

        headers = {
        "Content-Type": "application/json",
        "x-api-key": EXA_API_KEY
        }

        

        payload = {
            "query": query,
            "num_results": num_results,
            "include_domains": [],
            "exclude_domains": [],
            "use_autoprompt": True,   # optional for better queries
            "type": "auto"            
        }

        response = requests.post(EXA_ENDPOINT, json=payload, headers=headers,proxies={})
        response.raise_for_status()

        results = response.json()["results"]
        urls = [item["url"] for item in results]
        contents=[]
        headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
        }   
        
        for url in urls:
            response = requests.get(url,headers=headers)
            response.raise_for_status()  # raises HTTPError if status is 4xx or 5xx
            content = response.text
            contents.append(content)

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{query}.for the query that i just gave you i have searched the internet and i am going to provide you with the content of the three most relevant results.find the best answer for the query in the contents of these web sites  {contents}. just write the answer"}],
            max_tokens=100,
            temperature=0.1
        )
        
        response=response.choices[0].message.content.strip()

        return response
    
    
    def pdf_address_to_chunk_embedding(self,address,chunk_len=250):
        
        
       
        nltk.download('punkt_tab')
  

        doc = fitz.open(address)
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()

        # Step 2: Tokenize and chunk into 100-word chunks
        words = word_tokenize(full_text)
        chunks = [' '.join(words[i:i + chunk_len])
                  for i in range(0, len(words), chunk_len)]
        # compute embeddings as numpy float32 array
        embeddings = self.embedding_model.encode(chunks)
        embeddings = np.asarray(embeddings, dtype='float32')

        return chunks,embeddings
    


    def build_or_load_faiss_index(self,chunks, embeddings, index_path="faiss.index"):
       
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)                 # now a proper numpy array
        faiss.write_index(index, index_path)
        return index, chunks

    def search_faiss_index(self,index, chunks, query, top_k=3):

        q_emb = self.embedding_model.encode([query])
        q_emb = np.asarray(q_emb, dtype='float32')
        distances, indices = index.search(q_emb, top_k)
        print(indices)
        return [chunks[i] for i in indices[0]]

    def check_if_pdf_input(self,query):
        return '.pdf' in query.lower()
    
    def extract_local_pdf_paths(self,text):
        # Match file paths with spaces, until .pdf
        pattern = r'([A-Za-z]:[\\/](?:[^:*?"<>|\r\n]+[\\/])*[^:*?"<>|\r\n]+\.pdf)'
        matches = re.findall(pattern, text)
        pdf_paths = [match.strip() for match in matches if os.path.exists(match.strip())]
        return pdf_paths[0] if pdf_paths else None
    def game_mode(self,prompt):

        # response = self.client.chat.completions.create(
        #     model=self.model,
        #     messages=[{"role": "user", "content": f"Read this user input: -{prompt}-.If user input contain words like: game/20 question/20 question game/guess, just print : GAME"}],
        #     max_tokens=100,
        #     temperature=0.1
        # )
        
        # response=response.choices[0].message.content.strip()

        # if response=="GAME":
        #     return True
        
        # return False
        return '20 question game' in prompt.lower()


In [2]:
agent=Agent()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

c:\Users\PT\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PT\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some parameters are on the meta device because they were offloaded to the cpu.


# Sample outputs to show the Agent's abilities

1.ordinari conversation
2.reading the paper
3.searching the internet

In [ ]:
ans=agent.capture_the_flag('what is your model name?')
print(ans)

My model name is Mistral.


In [ ]:
ans=agent.capture_the_flag('How many parameters do you have?')
print(ans)

I have 7 parameters that control my behavior: learning rate, batch size, number of epochs, regularization strength, optimizer, activation function, and loss function.


In [ ]:
ans=agent.capture_the_flag('Who wrote the paper d://uni stuff//Semester 6//Deep Learning//Project//Large Language Diffusion Models.pdf')
print(ans)

The paper "Large Language Diffusion Models" was authored by Shen Nie, Fengqi Zhu, Zebin You, Xiaolu Zhang, Jingyang Ou, Jun Hu, Jun Zhou, Yankai Lin, and Ji-Rong Wen.


In [ ]:
ans=agent.capture_the_flag('Who scored in the last Real Madrid game?')
print(ans)

Kylian Mbappé got off to a perfect start in his second season with Real Madrid, leading the team to a 1-0 victory over Osasuna in their Spanish league opener Tuesday.


As you can see by changing the model to a local one the Agent work just as good as before.